In [1]:
import pandas as pd

This process notebook will investigate possible transformations to build a baseline model as start point, and from there we'll be improving the model, applying different techniques and feature engineering. 

In [2]:
# data
fraud_df = pd.read_csv("data/fraud_detection_dataset.csv")
fraud_df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


# Pre-processing Pipeline

- dropping columns
- filter categories: PAYMENT, DEBIT and CASH_IN. (*automatic tagged as not fraud*)
- training ans testing split
  - Is necessary that we keep the proportion of the target and also of the categorical variables
- label_encoder on type
- scale on numerical variables

In [4]:

url = "https://drive.google.com/u/0/uc?export=download&confirm=IHT2&id=1zjK8zQK5zvhR_r2chWI5dCjeOwASlPfb"
df = pd.read_csv(url)
df.head()

,"<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv=""content-type"" content=""text/html; charset=utf-8""/><link href=&#47;static&#47;doclist&#47;client&#47;css&#47;2674426593&#45;untrustedcontent.css rel=""stylesheet"" nonce=""GQZVnlG35O58fOLwu1XLZw""><link rel=""icon"" href=""//ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""/><style nonce=""GQZVnlG35O58fOLwu1XLZw"">#gbar",#guser{font-size:13px;padding-top:0px !important;}#gbar{height:22px}#guser{padding-bottom:7px !important;text-align:right}.gbh,.gbd{border-top:1px solid #c9d7f1;font-size:1px}.gbh{height:0;position:absolute;top:24px;width:100%}@media all{.gb1{height:22px;margin-right:.5em;vertical-align:top}#gbar{float:left}}a.gb1,a.gb4{text-decoration:underline !important}a.gb1,a.gb4{color:#00c !important}.gbi .gb4{color:#dd8e27 !important}.gbf .gb4{color:#900 !important}
0,"</style><script nonce=""XXWSg0iFr2IOv5DUf4fUAA""...",NaN,NaN,NaN,NaN


In [13]:
raw_url = "https://drive.google.com/file/d/1zjK8zQK5zvhR_r2chWI5dCjeOwASlPfb/view?usp=sharing"
url = "https://drive.google.com/ucexport=download&id=" + raw_url.split("/")[-2]

fraud_downloaded = pd.read_csv(url)
fraud_downloaded

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 3


In [3]:
from src.utils_preprocessing import PreProcessingPipe

In [4]:
pre_processing_pipe = PreProcessingPipe(dataset=fraud_df)
pre_processing_pipe.drop_columns(columns=["isFlaggedFraud", "step", "nameOrig", "nameDest"])
pre_processing_pipe.filter_type_classes(classes=["PAYMENT", "CASH_IN", "DEBIT"])
pre_processing_pipe.train_test_splitting(sample_test_size=0.40)

In [5]:
pre_processing_pipe.label_encoding()
pre_processing_pipe.X_train

['CASH_OUT', 'TRANSFER']


,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
4421351,1,3186916.14,383749.41,0.00,6082045.90,9268962.03
1677808,1,745013.00,0.00,0.00,4038650.74,4783663.74
454815,1,320697.68,102278.00,0.00,0.00,320697.68
694230,0,378854.34,51582.00,0.00,65230.97,444085.31
113562,0,367970.23,53708.00,0.00,258958.75,703945.69
...,...,...,...,...,...,...
6332630,0,3453.76,47105.00,43651.24,0.00,3453.76
1102634,0,391481.34,0.00,0.00,2490303.90,2881785.24
5698907,0,261398.34,9981.00,0.00,9771799.74,10033198.08
441011,0,104153.26,0.00,0.00,679988.08,784141.34


In [6]:
pre_processing_pipe.scaling()

In [7]:
pre_processing_pipe.X_train

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1.0,0.043169,0.006440,0.00000,0.017084,0.026023
1,1.0,0.010092,0.000000,0.00000,0.011344,0.013430
2,1.0,0.004344,0.001717,0.00000,0.000000,0.000900
3,0.0,0.005132,0.000866,0.00000,0.000183,0.001247
4,0.0,0.004984,0.000901,0.00000,0.000727,0.001976
...,...,...,...,...,...,...
1662240,0.0,0.000047,0.000791,0.00088,0.000000,0.000010
1662241,0.0,0.005303,0.000000,0.00000,0.006995,0.008091
1662242,0.0,0.003541,0.000168,0.00000,0.027448,0.028169
1662243,0.0,0.001411,0.000000,0.00000,0.001910,0.002202


# Training

In [8]:
from src.utils_preprocessing import Training

training_pipe = Training(
    X_train=pre_processing_pipe.X_train,
    X_test=pre_processing_pipe.X_test,
    y_train=pre_processing_pipe.y_train,
    y_test=pre_processing_pipe.y_test,
)
training_pipe.fit_logistic_regression()
training_pipe.predict_logistic_regression()
metrics = training_pipe.calculate_metrics()

metrics


/home/luba/.local/share/virtualenvs/TeraThridChallenge-Classification-_wtxcFRW/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'training': {'accuracy': (0.9972657460242023,),
  'recall': 0.08157467532467533,
  'precision': 0.9548693586698337,
  'auc': 0.5407816055060876},
 'testing': {'accuracy': (0.9972720644236773,),
  'recall': 0.08280060882800609,
  'precision': 0.9645390070921985,
  'auc': 0.5413957790315855}}

This is a very bad model, with an auc of 0.54. Is like the same thing of you try to guess if the transaction is fraud or not. But, the ideia was just to make a first base-model, to follow the process from start to finish with the api. 

In [9]:
import pickle

In [11]:
model_file_name = "models/lrc_baseline.sav"
pickle.dump(training_pipe.lrc, open(model_file_name, "wb"))

In [10]:
!mkdir models